<a href="https://colab.research.google.com/github/heomanism/Bioinformatics1/blob/main/Free_Proj/Bioinformatics1_FreeProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#생물정보학 자유 연구 주제

## 1. Fig S3C 재현해보기
## 2. LIN28A binding motif predictor 만들기
## 3. 2차구조를 같이 고려한 LIN28A binding motif predictor 만들기

### 2021-20784 허민(Min Heo)

## Google Drive 마운트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 환경 세팅

In [2]:
!git clone https://github.com/hyeshik/colab-biolab.git
!cd colab-biolab && bash tools/setup.sh
exec(open('colab-biolab/tools/activate_conda.py').read())

Cloning into 'colab-biolab'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 69 (delta 22), reused 54 (delta 13), pack-reused 0
Unpacking objects: 100% (69/69), done.
./
./root/
./root/.vimrc
./root/.bashrc.biolab
./root/.bin.priority/
./root/.bin.priority/pip3
./root/.bin.priority/pip
./root/.bin.priority/pip2
./root/.tmux.conf
./root/.condarc
./root/.profile
--2022-05-27 15:09:21--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76607678 (73M) [application/x-sh]
Saving to: ‘miniconda3.sh’

miniconda3.sh       100%[===================>]  73.06M  21.7MB/s    in 3.5s    

2022-05-27 15:09:25 (20.7 MB/s) - ‘miniconda3.sh’ sav

In [3]:
!conda install -y bedtools bioawk

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 4.13.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /root/conda/envs/lab

  added / updated specs:
    - bedtools
    - bioawk


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bioawk-1.0                 |       h7132678_7         196 KB  bioconda
    ------------------------------------------------------------
                                           Total:         196 KB

The following NEW packages will be INSTALLED:

  bioawk             bioconda/linux-64::bioawk-1.0-h7132678_7



bioawk-1.0           | 196 KB    | : 100% 1.0/1 [00:00<00:00,  4.97it/s]                
Preparing tran

In [4]:
%cd /content/drive/MyDrive/binfo1-work/

/content/drive/MyDrive/binfo1-work


In [5]:
! ls -al

total 8246684
-rw------- 1 root root 1370036258 May 19 12:05 CLIP-35L33G.bam
-rw------- 1 root root    3118336 May 19 12:05 CLIP-35L33G.bam.bai
-rw------- 1 root root       8692 May 19 14:29 CLIP-let7d.bam
-rw------- 1 root root      20809 May 19 14:48 CLIP-let7d-gene.pileup
-rw------- 1 root root      20809 May 19 14:29 CLIP-let7d.pileup
-rw------- 1 root root       7263 May 19 14:29 CLIP-let7f-1.bam
-rw------- 1 root root      15266 May 19 14:48 CLIP-let7f-1-gene.pileup
-rw------- 1 root root      17802 May 19 14:29 CLIP-let7f-1.pileup
-rw------- 1 root root       7077 May 19 14:27 CLIP-let7g.bam
-rw------- 1 root root      14561 May 19 12:28 CLIP-let7g-gene.pileup
-rw------- 1 root root    2685065 May 19 12:24 CLIP-let7g.pileup
-rw------- 1 root root     210519 May 27 08:48 CLIP-lin28a.bam
-rw------- 1 root root  402213545 May 13 03:52 filtered-RPF-siLin28a.bam
-rw------- 1 root root  567739792 May  6 04:26 filtered-RPF-siLuc.bam
-rw------- 1 root root   23136879 May 13 04:01 fivepc

In [6]:
## Lin28a를 포함하는 transcript 찾기 & 범위 알아보기
!cat gencode.gtf | awk '$3=="transcript" {print$0}' | grep -i lin28a

chr4	HAVANA	transcript	133730641	133746152	.	-	.	gene_id "ENSMUSG00000050966.10"; transcript_id "ENSMUST00000051674.3"; gene_type "protein_coding"; gene_name "Lin28a"; transcript_type "protein_coding"; transcript_name "Lin28a-201"; level 2; protein_id "ENSMUSP00000050488.3"; transcript_support_level "1"; mgi_id "MGI:1890546"; tag "basic"; tag "appris_principal_1"; tag "CCDS"; ccdsid "CCDS18761.1"; havana_gene "OTTMUSG00000010211.3"; havana_transcript "OTTMUST00000023733.1";
chr4	HAVANA	transcript	133733555	133739697	.	-	.	gene_id "ENSMUSG00000050966.10"; transcript_id "ENSMUST00000176113.8"; gene_type "protein_coding"; gene_name "Lin28a"; transcript_type "protein_coding"; transcript_name "Lin28a-202"; level 2; protein_id "ENSMUSP00000135254.2"; transcript_support_level "5"; mgi_id "MGI:1890546"; tag "mRNA_end_NF"; tag "cds_end_NF"; havana_gene "OTTMUSG00000010211.3"; havana_transcript "OTTMUST00000104013.2";
chr4	HAVANA	transcript	133733561	133736722	.	-	.	gene_id "ENSMUSG00000050966.1

In [7]:
## 해당 좌표 범위로 CLIP-lin28a.bam 파일 만들기
!samtools view -b -o CLIP-lin28a.bam CLIP-35L33G.bam chr4:133730641-133746152 chr4:133733555-133739697 chr4:133733561-133736722 chr4:133735259-133745693
!samtools view CLIP-lin28a.bam | wc -l

5349


In [13]:
! samtools sort CLIP-lin28a.bam -o Sorted_CLIP-lin28a.bam

In [14]:
## samtools mpileup for watching read's distribution
!samtools mpileup Sorted_CLIP-lin28a.bam > CLIP-lin28a.pileup
!wc -l CLIP-lin28a.pileup

[mpileup] 1 samples in 1 input files
144525 CLIP-lin28a.pileup


In [18]:
!head CLIP-lin28a.pileup

chr4	133655662	N	4	^It^It^It^Gt	I<I:
chr4	133655663	N	4	tttt	IFIB
chr4	133655664	N	4	tttt	IHIE
chr4	133655665	N	4	tttt	IDIE
chr4	133655666	N	4	cccc	IIID
chr4	133655667	N	4	cccc	IIIE
chr4	133655668	N	4	aaaa	IGIB
chr4	133655669	N	4	gggg	IIIG
chr4	133655670	N	4	<<<<	IGIG
chr4	133655671	N	4	<<<<	IGIG


In [19]:
!awk '($2 >= 133730641 && $2 <= 133746152) || ($2 >= 133733555 && $2 <= 133739697) || ($2 >= 133733561 && $2 <= 133736722) || ($2 >= 133735259 && $2 <= 133745693)   { print $0; }' CLIP-lin28a.pileup > CLIP-lin28a-gene.pileup
!tail CLIP-lin28a-gene.pileup

chr4	133746143	N	48	<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<	IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID
chr4	133746144	N	48	<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<	IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID
chr4	133746145	N	48	<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<	IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID
chr4	133746146	N	48	<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<	IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID
chr4	133746147	N	48	<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<	IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID
chr4	133746148	N	48	<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<	IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID
chr4	133746149	N	48	<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<	IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID
chr4	133746150	N	48	<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<	IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID
chr4	133746151	N	48	<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

In [20]:
import pandas as pd

lin28a_pileup = pd.read_csv('CLIP-lin28a-gene.pileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'])
lin28a_pileup

,chrom,pos,_ref,count,basereads,quals
0,chr4,133730641,N,49,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IGIGGIH=EHGIIH=EHGIIH=EHGIIH=EHGIHIDBHIDBHIDBHIDB
1,chr4,133730642,N,49,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IGIGGIH=EHGIIH=EHGIIH=EHGIIH=EHGIHIDBHIDBHIDBHIDB
2,chr4,133730643,N,49,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IGIGGIH=EHGIIH=EHGIIH=EHGIIH=EHGIHIDBHIDBHIDBHIDB
3,chr4,133730644,N,49,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IGIGGIH=EHGIIH=EHGIIH=EHGIIH=EHGIHIDBHIDBHIDBHIDB
4,chr4,133730645,N,49,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IGIGGIH=EHGIIH=EHGIIH=EHGIIH=EHGIHIDBHIDBHIDBHIDB
...,...,...,...,...,...,...
15507,chr4,133746148,N,48,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID
15508,chr4,133746149,N,48,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID
15509,chr4,133746150,N,48,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID
15510,chr4,133746151,N,48,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID


In [21]:
import re
toremove = re.compile('[<>$*#]')  ## 이렇게 하는 것이 맞는게 아닐까?
toremove2= re.compile('\^.')  ## 이렇게 하는 것이 맞는게 아닐까?
lin28a_pileup['matches'] = lin28a_pileup['basereads'].apply(lambda x: toremove.sub('', x))
lin28a_pileup['matches'] = lin28a_pileup['matches'].apply(lambda x: toremove2.sub('', x))
lin28a_pileup

,chrom,pos,_ref,count,basereads,quals,matches
0,chr4,133730641,N,49,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IGIGGIH=EHGIIH=EHGIIH=EHGIIH=EHGIHIDBHIDBHIDBHIDB,
1,chr4,133730642,N,49,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IGIGGIH=EHGIIH=EHGIIH=EHGIIH=EHGIHIDBHIDBHIDBHIDB,
2,chr4,133730643,N,49,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IGIGGIH=EHGIIH=EHGIIH=EHGIIH=EHGIHIDBHIDBHIDBHIDB,
3,chr4,133730644,N,49,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IGIGGIH=EHGIIH=EHGIIH=EHGIIH=EHGIHIDBHIDBHIDBHIDB,
4,chr4,133730645,N,49,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IGIGGIH=EHGIIH=EHGIIH=EHGIIH=EHGIHIDBHIDBHIDBHIDB,
...,...,...,...,...,...,...,...
15507,chr4,133746148,N,48,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID,
15508,chr4,133746149,N,48,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID,
15509,chr4,133746150,N,48,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID,
15510,chr4,133746151,N,48,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID,


In [23]:
lin28a_pileup[['chrom', 'pos','_ref','matches']]

,chrom,pos,_ref,matches
0,chr4,133730641,N,
1,chr4,133730642,N,
2,chr4,133730643,N,
3,chr4,133730644,N,
4,chr4,133730645,N,
...,...,...,...,...
15507,chr4,133746148,N,
15508,chr4,133746149,N,
15509,chr4,133746150,N,
15510,chr4,133746151,N,


In [26]:
## Make Entropy calculation function for genome
import collections
 
from scipy.stats import entropy
  
def estimate_shannon_entropy(dna_sequence):
    bases = collections.Counter([tmp_base for tmp_base in dna_sequence])
    # define distribution
    dist = [x/sum(bases.values()) for x in bases.values()]
 
    # use scipy to calculate entropy
    entropy_value = entropy(dist, base=2)
 
    return entropy_value

In [27]:
lin28a_pileup['entropy'] = lin28a_pileup['matches'].map(lambda x : estimate_shannon_entropy(x))
lin28a_pileup['entropy']

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
15507    0.0
15508    0.0
15509    0.0
15510    0.0
15511    0.0
Name: entropy, Length: 15512, dtype: float64

In [28]:
bedgraph_lin28a = lin28a_pileup[['chrom','pos','pos','entropy']].copy()
bedgraph_lin28a.columns = ['chrom','chromStart','chromEnd','dataValue']

In [29]:
bedgraph_lin28a.to_csv('lin28a.BedGraph', index=False,sep=' ') 